In [4]:
import requests
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta
import csv, os, re, time
from selenium import webdriver
from bs4 import BeautifulSoup
import csv
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [2]:
url = "https://www.eppo.go.th/epposite/index.php/th/petroleum/price/oil-price?orders%5BpublishUp%5D=publishUp&issearch=1"
html_page = requests.get(url)

print(html_page)
print(type(html_page))


<Response [200]>
<class 'requests.models.Response'>


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

url = "http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php"

driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)  # รอโหลด

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

oil_div = soup.find("div", class_="div_oil_price")
print(oil_div is not None)  


True


## วันปัจจุบัน

In [4]:
url = "http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php"

driver = webdriver.Chrome()
driver.get(url)
time.sleep(2)

soup = BeautifulSoup(driver.page_source, "html.parser")
driver.quit()

oil_div = soup.find("div", class_="div_oil_price")

# ===== ดึงวันที่จาก header =====
date_header = oil_div.find("div", style=re.compile("font-size: 2em"))
raw_date = date_header.get_text(strip=True).replace("Oil price", "").strip()
date = datetime.strptime(raw_date, "%d %b %Y").date()

# ===== หา Gasohol 95 =====
g95_img = oil_div.find("img", {"src": re.compile("oil_name2.png")})
g95_row = g95_img.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
g95_prices = [col.text.strip() for col in g95_row.find_all("div", class_="oil_price_colum")]

# ===== หา Effective date =====
eff_label = None
for div in oil_div.find_all("div", class_="oil_price_colum_name"):
    if "Effective date" in div.get_text():
        eff_label = div
        break

eff_row = eff_label.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
eff_dates = [col.text.strip() for col in eff_row.find_all("div", class_="oil_price_colum")]

# ===== Mapping 3 ปั๊ม =====
stations = ["PTT", "Bangchak", "Shell"]
indices = [0, 1, 2]

result = {
    "date": date,
    "data": {
        station: {
            "price_gasohol_95": g95_prices[i],
            "effective_date": eff_dates[i]
        }
        for station, i in zip(stations, indices)
    }
}

print(result)

# วันนี้แบบไม่ใช้ API

{'date': datetime.date(2025, 9, 7), 'data': {'PTT': {'price_gasohol_95': '32.95', 'effective_date': '22 Aug 05:00'}, 'Bangchak': {'price_gasohol_95': '32.95', 'effective_date': '22 Aug 05:00'}, 'Shell': {'price_gasohol_95': '33.45', 'effective_date': '22 Aug 05:00'}}}


In [5]:
rows = []
for station, i in zip(stations, indices):
    rows.append([
        date,          # วันที่ scrape
        station,             # ชื่อปั๊ม
        g95_prices[i],       # ราคาน้ำมัน Gasohol 95
        eff_dates[i]         # Effective date
    ])

# ===== บันทึก CSV =====
with open("oil_prices.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["date", "station", "price_gasohol_95", "effective_date"])
    writer.writerows(rows)

print("✅ บันทึกข้อมูลลงไฟล์ oil_prices.csv เรียบร้อยแล้ว")

✅ บันทึกข้อมูลลงไฟล์ oil_prices.csv เรียบร้อยแล้ว


## ย้อนหลัง 3 วัน

In [ ]:
def scrape_price(date_str):
    """Scrape ราคาน้ำมันจาก eppo_oil_gen_new.php ตามวันที่ที่กำหนด (dd/mm/yyyy)"""
    url = f"http://www.eppo.go.th/epposite/templates/eppo_v15_mixed/eppo_oil/eppo_oil_gen_new.php?TbxToDate={date_str}"
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    oil_div = soup.find("div", class_="div_oil_price")

    # ===== หา Gasohol 95 =====
    g95_img = oil_div.find("img", {"src": re.compile("oil_name2.png")})
    g95_row = g95_img.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
    g95_prices = [col.text.strip() for col in g95_row.find_all("div", class_="oil_price_colum")]

    # ===== หา Effective date =====
    eff_label = None
    for div in oil_div.find_all("div", class_="oil_price_colum_name"):
        if "Effective date" in div.get_text():
            eff_label = div
            break
    eff_row = eff_label.find_parent("div", class_=re.compile("oil_price_colum_name_(odd|even)"))
    eff_dates = [col.text.strip() for col in eff_row.find_all("div", class_="oil_price_colum")]

    stations = ["PTT", "Bangchak", "Shell"]
    indices = [0, 1, 2]

    # แปลง date เป็น ISO
    date = datetime.strptime(date_str, "%d/%m/%Y").strftime("%Y-%m-%d")
    year = date.split("-")[0]

    # แปลง effective_date
    eff_dates_iso = []
    for eff in eff_dates:
        if eff == "-":
            eff_dates_iso.append("")
        else:
            eff_dt = datetime.strptime(f"{eff} {year}", "%d %b %H:%M %Y")
            eff_dates_iso.append(eff_dt.strftime("%Y-%m-%d %H:%M"))

    result = []
    for station, i in zip(stations, indices):
        result.append({
            "date": date,
            "station": station,
            "price_gasohol_95": g95_prices[i],
            "effective_date": eff_dates_iso[i]
        })
    return result


# ====== scrap ย้อนหลัง 3 วันจาก "วันนี้" ======
today = date.today()   # ✅ ใช้วันที่ปัจจุบันแบบ real time
all_data = []

for i in range(1,4):
    day = today - timedelta(days=i)
    date_str = day.strftime("%d/%m/%Y")
    all_data.extend(scrape_price(date_str))


In [ ]:
# ====== บันทึก CSV ======
file_name = "oil_prices.csv"
file_exists = os.path.isfile(file_name)

with open(file_name, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "price_gasohol_95", "effective_date"])
    if not file_exists:
        writer.writeheader()
    writer.writerows(all_data)

print(f"✅ เก็บข้อมูล {len(all_data)} แถว จาก {today} ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว")

✅ เก็บข้อมูล 9 แถว จาก 2025-09-07 ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว


## ย้อนหลัง 3 เดือน

In [21]:
import os
import time
import glob
import csv
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

# === ตั้งค่าโฟลเดอร์ดาวน์โหลด ===
download_dir = os.path.abspath("downloads")
os.makedirs(download_dir, exist_ok=True)

options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
options.add_experimental_option("prefs", prefs)
options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=options)

# URL หลัก
base_url = "https://www.eppo.go.th/epposite/index.php/th/petroleum/price/oil-price?orders[publishUp]=publishUp&issearch=1"

# === ไฟล์ CSV output ===
output_file = "oil_price_output.csv"
with open(output_file, mode="w", newline="", encoding="utf-8-sig") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "price_gasohol_95", "effective_date"])
    writer.writeheader()

    # วนย้อนหลัง 3 วัน
    start_date = datetime.today() - timedelta(days=3)
    end_date = datetime.today()

    for d in range((end_date - start_date).days + 1):
        date = (start_date + timedelta(days=d)).strftime("%d/%m/%Y")
        print(f"📌 Processing date: {date}")

        driver.get(base_url)
        time.sleep(3)  # ให้หน้า render iframe

        # === ตรวจสอบ iframe และหา TbxToDate ===
        iframes = driver.find_elements(By.TAG_NAME, "iframe")
        found = False
        for i, iframe in enumerate(iframes):
            driver.switch_to.frame(iframe)
            try:
                date_box = WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.ID, "TbxToDate"))
                )
                print(f"✅ เจอ TbxToDate ใน iframe {i}")
                found = True
                break
            except:
                driver.switch_to.default_content()

        if not found:
            print("❌ ไม่เจอช่องใส่วันที่")
            continue

        # ✅ กรอกวันที่
        date_box.clear()
        date_box.send_keys(date)

        # ✅ กดปุ่ม Generate
        btn = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.ID, "BtnGenerate"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", btn)
        time.sleep(1)
        try:
            btn.click()
        except:
            driver.execute_script("arguments[0].click();", btn)

        # === CASE 1: ตาราง ===
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CLASS_NAME, "div_oil_price"))
            )
            soup = BeautifulSoup(driver.page_source, "html.parser")
            oil_div = soup.find("div", class_="div_oil_price")

            if oil_div:
                table = oil_div.find("table")
                rows = table.find_all("tr")

                # header = station
                header_row = rows[0].find_all("td")[1:]
                stations = [h.get("title", h.get_text(strip=True)) for h in header_row]

                # row = Gasohol 95
                g95_row = None
                for row in rows:
                    if "Gasohol 95" in row.get_text():
                        g95_row = row.find_all("td")[1:]
                        break
                g95_prices = [c.get_text(strip=True) for c in g95_row]

                # effective date
                eff_row = rows[-1].find_all("td")[1:]
                eff_dates = [c.get_text(strip=True) for c in eff_row]

                for st, pr, eff in zip(stations, g95_prices, eff_dates):
                    writer.writerow({
                        "date": datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d"),
                        "station": st,
                        "price_gasohol_95": pr,
                        "effective_date": eff
                    })
                print(f"✅ Scraped table for {date}")
                driver.switch_to.default_content()
                continue

        except:
            print("⚠️ ไม่เจอตาราง ลองตรวจสอบไฟล์ดาวน์โหลด...")

        # === CASE 2: ไฟล์ดาวน์โหลด ===
        driver.switch_to.default_content()
        file_path = None
        timeout = 180  # รอสูงสุด 3 นาที
        start_wait = time.time()
        while time.time() - start_wait < timeout:
            files = glob.glob(os.path.join(download_dir, "*.xls*"))
            if files:
                file_path = max(files, key=os.path.getctime)
                if not file_path.endswith(".crdownload"):
                    # เช็กว่าโหลดเสร็จจริง (ขนาดไฟล์หยุดเปลี่ยน)
                    prev_size = -1
                    while True:
                        curr_size = os.path.getsize(file_path)
                        if curr_size == prev_size:
                            break
                        prev_size = curr_size
                        time.sleep(2)
                    break
            time.sleep(2)

        if not file_path:
            print(f"❌ ไม่มีไฟล์ดาวน์โหลดของ {date}")
            continue

        print(f"✅ เจอไฟล์ดาวน์โหลด: {file_path}")

        # อ่าน Excel (เลือก engine อัตโนมัติ)
        if file_path.endswith(".xls"):
            df = pd.read_excel(file_path, engine="xlrd", header=None)
        else:
            df = pd.read_excel(file_path, engine="openpyxl", header=None)

        # --- parse ไฟล์ EPPO ---
        # row แรก (index=0) = header (เว้นคอลัมน์แรกไว้)
        stations = list(df.iloc[0, 1:])

        # หา row ที่มี Gasohol 95
        g95_row = df[df.iloc[:, 0].astype(str).str.contains("Gasohol 95", na=False)]
        if g95_row.empty:
            print(f"⚠️ No Gasohol 95 row in {file_path}")
            continue
        g95_prices = list(g95_row.values[0][1:])

        # row ล่างสุด = effective_date
        eff_dates = list(df.iloc[-1, 1:])

        for st, pr, eff in zip(stations, g95_prices, eff_dates):
            if pd.isna(pr):
                continue
            writer.writerow({
                "date": datetime.strptime(date, "%d/%m/%Y").strftime("%Y-%m-%d"),
                "station": st,
                "price_gasohol_95": pr,
                "effective_date": eff
            })

driver.quit()
print(f"🎉 Done! All results saved to {output_file}")


📌 Processing date: 04/09/2025
✅ เจอ TbxToDate ใน iframe 1
⚠️ ไม่เจอตาราง ลองตรวจสอบไฟล์ดาวน์โหลด...
❌ ไม่มีไฟล์ดาวน์โหลดของ 04/09/2025
📌 Processing date: 05/09/2025
✅ เจอ TbxToDate ใน iframe 1
⚠️ ไม่เจอตาราง ลองตรวจสอบไฟล์ดาวน์โหลด...
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\Oil-Prediction\downloads\EPPO_RetailOilPrice_on_20250905.xls
📌 Processing date: 06/09/2025
✅ เจอ TbxToDate ใน iframe 1
⚠️ ไม่เจอตาราง ลองตรวจสอบไฟล์ดาวน์โหลด...
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\Oil-Prediction\downloads\EPPO_RetailOilPrice_on_20250906.xls
📌 Processing date: 07/09/2025
✅ เจอ TbxToDate ใน iframe 1
⚠️ ไม่เจอตาราง ลองตรวจสอบไฟล์ดาวน์โหลด...
✅ เจอไฟล์ดาวน์โหลด: c:\zzz\Oil-Prediction\downloads\EPPO_RetailOilPrice_on_20250907.xls
🎉 Done! All results saved to oil_price_output.csv


In [8]:
# ====== บันทึก CSV ======
file_name = "oil_prices.csv"
file_exists = os.path.isfile(file_name)

with open(file_name, "a", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["date", "station", "price_gasohol_95", "effective_date"])
    if not file_exists:
        writer.writeheader()
    writer.writerows(all_data)

print(f"✅ เก็บข้อมูล {len(all_data)} แถว จาก {today} ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว")

✅ เก็บข้อมูล 279 แถว จาก 2025-09-07 ย้อนหลัง 3 วัน บันทึกลง oil_prices.csv แล้ว
